In [29]:
import pandas as pd
from psycopg2 import connect
from sqlalchemy import create_engine

# class Configuration:
#     PG_HOST = 'business-intelligent-db.cw1neqwhyrda.eu-central-1.rds.amazonaws.com'
#     PG_USER = 'postgres'
#     PG_PWD = '52eXzbKbqmY2x45f'

PROCESSING_FILE_NAME = '1989.csv'
DB_NAME = 'airline'
TABLE_NAME = 'flight'

class Configuration:
    PG_HOST = 'localhost'
    PG_USER = 'postgres'
    PG_PWD = 'gherukdnbi6lrn4ieljb549ew9j4v98le5j9v48vl5ee'
    
engine = create_engine(f"postgresql://{Configuration.PG_USER}:{Configuration.PG_PWD}@{Configuration.PG_HOST}:5432/{DB_NAME}")
conn = engine.connect()
psy_conn = connect(host=Configuration.PG_HOST,
                   user=Configuration.PG_USER,
                   password=Configuration.PG_PWD,
                   database="airline",
                   port=5432)


In [20]:
import os
import psycopg2
# Not correct way to copy to pg db table
def copy_from_file(conn, table):
    # save the dataframe on disk as a csv file, load
    # the csv file and use copy_from() to copy it to the table
    tmp_df = f"./result/{PROCESSING_FILE_NAME}"
    # df.to_csv(tmp_df, header=True, index=False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=',')
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        # os.remove(tmp_df)
        print("Error: %s" %error)
        conn.rollback()
        conn.close()
        return 1
    print("copy_from_file() done")
    cursor.close()
    # os.remove(tmp_df)

In [8]:
# connect to our postgres DB
conn = connect(host=Configuration.PG_HOST,
                user=Configuration.PG_USER,
                password=Configuration.PG_PWD,
                database="airline",
                port=5432)
copy_from_file(conn, 'flight')

Error: invalid input syntax for type integer: "year"
CONTEXT:  COPY flight, line 1, column year: "year"



1

## Load data from csv (corrected way)

In [30]:
processed_flight_df = f"./result/{PROCESSING_FILE_NAME}"
flight_df = pd.read_csv(processed_flight_df)
flight_df.dtypes

year                     int64
month                    int64
day_of_month             int64
day_of_week              int64
dep_time                object
crs_dep_time            object
arr_time                object
crs_arr_time            object
unique_carrier          object
flight_num               int64
actual_elapsed_time    float64
crs_elapsed_time         int64
arr_delay              float64
dep_delay              float64
origin                  object
dest                    object
distance               float64
cancelled                int64
diverted                 int64
dtype: object

In [31]:
def convert_df_to_db_data_type(df):
    df['cancelled'] = df['cancelled'].astype(bool)
    df['diverted'] = df['diverted'].astype(bool)

In [33]:
# convert pandas df to db record data type
convert_df_to_db_data_type(flight_df)
flight_df.dtypes

year                     int64
month                    int64
day_of_month             int64
day_of_week              int64
dep_time                object
crs_dep_time            object
arr_time                object
crs_arr_time            object
unique_carrier          object
flight_num               int64
actual_elapsed_time    float64
crs_elapsed_time         int64
arr_delay              float64
dep_delay              float64
origin                  object
dest                    object
distance               float64
cancelled                 bool
diverted                  bool
dtype: object

In [34]:
# Save df to db
flight_df.to_sql(TABLE_NAME, engine, if_exists='append', index=False)

psy_conn.autocommit = True
cursor = psy_conn.cursor()
  
sql1 = 'SELECT count(*) from flight;'
cursor.execute(sql1)
for i in cursor.fetchall():
    print(i)

# conn.commit()
conn.close()

(9890576,)
